<a href="https://colab.research.google.com/github/bhkpriyanka/EVA4/blob/master/Week4/S4_AssignmentSolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #input -? OUtput? RF
        self.conv1 = nn.Conv2d(1, 12, 3, padding=1, bias = False) 
        self.bn1   = nn.BatchNorm2d(12)
        self.d1    = nn.Dropout2d(0.02)
        
        self.conv2 = nn.Conv2d(12, 16, 3, padding=1, bias = False)
        self.bn2   = nn.BatchNorm2d(16)
        self.d2    = nn.Dropout2d(0.02)

        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(16, 24, 3, padding=1, bias = False )
        self.bn3   = nn.BatchNorm2d(24)
        self.d3    = nn.Dropout2d(0.04)

        self.conv4 = nn.Conv2d(24, 24, 3, padding=1, bias = False)
        self.bn4   = nn.BatchNorm2d(24)
        self.d4    = nn.Dropout2d(0.04)

        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(24, 40, 3, bias = False)
        self.bn5   = nn.BatchNorm2d(40)
        self.d5    = nn.Dropout2d(0.06)

        self.conv6 = nn.Conv2d(40,10,1, bias =False)
        
        self.gap1 = nn.AdaptiveAvgPool3d((10,1,1))

        self.conv7_fc = nn.Linear(10,10)
        

    def forward(self, x):
        x = self.pool1(self.d2(self.bn2(F.relu(self.conv2(self.d1(self.bn1(F.relu(self.conv1(x)))))))))
        x = self.pool2(self.d4(self.bn4(F.relu(self.conv4(self.d3(self.bn3(F.relu(self.conv3(x)))))))))
        x = self.gap1(self.conv6(self.d5(self.bn5(F.relu(self.conv5(x))))))
        x = x.view(-1, 10)
        x = self.conv7_fc(x)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 12, 28, 28]             108
       BatchNorm2d-2           [-1, 12, 28, 28]              24
         Dropout2d-3           [-1, 12, 28, 28]               0
            Conv2d-4           [-1, 16, 28, 28]           1,728
       BatchNorm2d-5           [-1, 16, 28, 28]              32
         Dropout2d-6           [-1, 16, 28, 28]               0
         MaxPool2d-7           [-1, 16, 14, 14]               0
            Conv2d-8           [-1, 24, 14, 14]           3,456
       BatchNorm2d-9           [-1, 24, 14, 14]              48
        Dropout2d-10           [-1, 24, 14, 14]               0
           Conv2d-11           [-1, 24, 14, 14]           5,184
      BatchNorm2d-12           [-1, 24, 14, 14]              48
        Dropout2d-13           [-1, 24, 14, 14]               0
        MaxPool2d-14             [-1, 2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0
    correct1 = 0
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        #bhjh
        train_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
        pred1 = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct1 += pred1.eq(target.view_as(pred1)).sum().item()
        #gdg
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

    train_loss /= len(train_loader.dataset)

    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        train_loss, correct1, len(train_loader.dataset),
        100. * correct1 / len(train_loader.dataset)))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    print('EPOCH: ',epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

EPOCH:  1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.1155146136879921 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 34.37it/s]


Train set: Average loss: 0.5799, Accuracy: 48787/60000 (81.31%)




  0%|          | 0/469 [00:00<?, ?it/s]

...


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0177, Accuracy: 9942/10000 (99.42%)

EPOCH:  20


loss=0.0056805857457220554 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 40.85it/s]


Train set: Average loss: 0.0159, Accuracy: 59692/60000 (99.49%)




Test set: Average loss: 0.0170, Accuracy: 9948/10000 (99.48%)

